DNN but with the 50:50 ratio between SM e BSM enforced with weights

In [ ]:
from glob import glob
import sys, scipy, pickle
from scipy.stats import chi2, poisson
import numpy as np

import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
matplotlib.rcParams.update({'font.size': 16})

from prettytable import PrettyTable

In [ ]:
import ROOT as rt
import root_numpy as rtnp
from utility_func import *

In [ ]:
from keras.layers import Input, Dense, Lambda, Dropout
from keras.models import Model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, TerminateOnNaN, ModelCheckpoint
from keras.constraints import max_norm
from keras.models import load_model
import keras
print keras.__version__

In [ ]:
do_training = 1

# Get the samples

In [ ]:
hlf_features = ['HT', 'METp', 'METo', 'MT', 'nJets', 
                'bJets', 'allJetMass', 'LepPt', 'LepEta', 
                'LepIsoCh', 'LepIsoGamma', 'LepIsoNeu', 'LepCharge', 
                'LepIsEle', 'nMu', 'allMuMass', 'allMuPt', 'nEle', 
                'allEleMass', 'allElePt', 'nChHad', 'nNeuHad', 'nPhoton']

active_hlf_features = ['HT', 'allJetMass',
                       'allMuPt', 'allMuMass',
                       'allElePt','allEleMass',
                       'LepPt', 'LepIsoCh', 'LepIsoGamma', 'LepIsoNeu',
                       
                       'METp', 'METo',
                       
                       'MT',
                       
                       'nMu', 'nJets', 'bJets', 'nEle', 
                       
                       'LepCharge','LepIsEle', 
                       
                       'nChHad', 'nNeuHad']

Nf_lognorm = 10
Nf_gauss = 2
Nf_Pgauss = 1
Nf_PDgauss = 4
Nf_binomial = 2
Nf_poisson = 2

sel = []
for a in active_hlf_features:
    for i,f in enumerate(hlf_features):
        if a == f:
            sel.append(i)
# print 'Activated HLF:'
# for n in np.array(hlf_features)[sel]:
#     print '\t', n

In [ ]:
data_folder = '../data/HLF_ONLY/'
TrainSamplesName = ['Wlnu', 'qcd', 'Zll', 'ttbar']
BSM_samples = ['Ato4l', 'Zprime', 'Wprime', 'leptoquark', 'hToTauTau', 'hChToTauNu']

N_train_max = int(9e6)
training_split_fraction = 0.5

In [ ]:
fraction = np.array([0.592, 0.338, 0.067, 0.003])

raw_sample = {}
l = np.zeros(4)
for i,n in enumerate(TrainSamplesName):
    raw_sample[n] = np.load(data_folder+n+'_sample.npy')
#     np.random.shuffle(raw_sample[n])
    l[i] = raw_sample[n].shape[0]
    
i_min = np.argmin(l/fraction)
if TrainSamplesName[i_min]=='qcd':
    print 'QCD is limiting, using it for both val and split'
    N_train = min(N_train_max, l[i_min]/fraction[i_min])
else:
    N_train = min(N_train_max, training_split_fraction*l[i_min]/fraction[i_min])
    
if N_train < N_train_max:
    print 'Limiting stat. sample:', TrainSamplesName[i_min]
else:
    print 'Sample available satisfying '

N_val = N_train*(1-training_split_fraction)/training_split_fraction - 1
print 'Expected {:.2f} train'.format(N_train/1.0e6)
print 'Expected {:.2f} val'.format(N_val/1.0e6)

In [ ]:
x_train_s = {}
x_val_s = {}

table = PrettyTable(['Sample', 'Evts tot', 'Train', 'Val'])

for i,n in enumerate(TrainSamplesName):
    N_train_aux = int(N_train * fraction[i])
    x_train_s[n] = raw_sample[n][:N_train_aux, sel]
    N_val_aux = int(N_val * fraction[i])
    if TrainSamplesName[i_min]=='qcd' and n == 'qcd':
        print 'QCD is limiting, using it for both val and split'
        np.random.shuffle(raw_sample[n])
        x_val_s[n] = raw_sample[n][:N_val_aux, sel]
    elif N_train_aux+N_val_aux < raw_sample[n].shape[0]:
        x_val_s[n] = raw_sample[n][N_train_aux : N_train_aux+N_val_aux, sel]
    else:
        print 'Error', n
        continue
    table.add_row([n, raw_sample[n].shape[0], x_train_s[n].shape[0], x_val_s[n].shape[0]])
print table
    
x_train = np.concatenate((x_train_s['Wlnu'], x_train_s['qcd'], x_train_s['Zll'], x_train_s['ttbar']))
x_val = np.concatenate((x_val_s['Wlnu'], x_val_s['qcd'], x_val_s['Zll'], x_val_s['ttbar']))

print 'Tot training {:.2f} M'.format(x_train.shape[0]/1.0e6)
print 'Tot val {:.2f} M'.format(x_val.shape[0]/1.0e6)

In [ ]:
x_sig_train = {}
x_sig_val = {}
weight_sig = {}
for n in BSM_samples:
    s = np.load(data_folder+n+'_sample.npy')
    N_aux = min(int(N_train_max/training_split_fraction), s.shape[0])
    N_train_aux = int(N_aux*training_split_fraction)
    N_val_aux = N_aux - N_train_aux
    x_sig_train[n] = s[:N_train_aux, sel]
    x_sig_val[n] = s[N_train_aux:N_train_aux+N_val_aux, sel]
    w = float(x_train.shape[0])/x_sig_train[n].shape[0]
    weight_sig[n] = w
    print '{} {:.3f}M ({:.1f})'.format(n, s.shape[0]*1.e-6, w)

# Start the Classifier declaration

In [ ]:
original_dim = x_train.shape[1]
act_fun = 'relu'
clf = {}

In [ ]:
global w_BSM
w_BSM = 1.0

def weighted_binary_xentropy(y_true, y_pred):
    print w_BSM
    aux = K.binary_crossentropy(y_true, y_pred)

    w = K.tf.where(K.tf.greater(y_true, 0.5), K.tf.ones_like(y_true)*w_BSM, K.tf.ones_like(y_true))
    return K.dot(K.transpose(w), aux)/K.sum(w)

In [ ]:
def plot_history(h, tag='', save_values=False):
    f = plt.figure(figsize=(16,8))
    plt.plot(h['loss'][1:], '.-', label='loss')
    plt.plot(h['val_loss'][1:], '.--', label='val_loss')

    plt.xlabel('Epoch')
    plt.grid()
    plt.legend(loc='best')
    plt.yscale('log')
    if tag:
        f.savefig('Classifier_HLF_v3/{}_training_hisotry.png'.format(tag))
        if save_values:
            f = open('Classifier_HLF_v3/{}_training_hisotry.pkl'.format(tag), 'w')
            pickle.dump(h, f, pickle.HIGHEST_PROTOCOL)
            f.close()

In [ ]:
def train_it(model, n, ES_p=30, RLRP_p=10, epochs=100):
    clf[n] = model

    global w_BSM
    w_BSM = weight_sig[n]
    clf[n].compile(loss=weighted_binary_xentropy, optimizer='adam')
#     clf[n].compile(loss='binary_crossentropy', optimizer='adam')

    print '\n\n------------------ Training', n, '--------------------'
    aux_x = np.concatenate(( x_train, x_sig_train[n] ))
    aux_y = np.concatenate(( np.zeros(x_train.shape[0]), np.ones(x_sig_train[n].shape[0]) ))
    aux_xval = np.concatenate(( x_val, x_sig_val[n] ))
    aux_yval = np.concatenate(( np.zeros(x_val.shape[0]), np.ones(x_sig_val[n].shape[0]) ))

    clf[n].report = clf[n].fit(x=aux_x, y=aux_y, batch_size=500, epochs=epochs,
                          shuffle=True,
                          validation_data= (aux_xval, aux_yval),
                          callbacks = [
                                        EarlyStopping(monitor='val_loss', patience=ES_p, verbose=1),
                                        ReduceLROnPlateau(monitor='val_loss', patience=RLRP_p, verbose=1),
                                        TerminateOnNaN(),
                                        ModelCheckpoint('Classifier_HLF_v3/Clf_{}_best.hdf5'.format(n), 
                                                    monitor='val_loss',
                                                    save_best_only=True, 
                                                    save_weights_only=True)
                                      ]
                         )
    plot_history(clf[n].report.history, n, True)

### Do the training

In [ ]:
n = 'Ato4l'

if do_training:
    inputs = Input(shape=(original_dim,), name='Input_'+n)
    mdl = Dense(20, activation=act_fun, name='h1_'+n)(inputs)
    #     mdl = Dropout(0.2)(mdl)
    mdl = Dense(20, activation=act_fun, name='h2_'+n)(mdl)
#     mdl = Dropout(0.1)(mdl)
    mdl = Dense(1, activation='sigmoid', name='out_'+n)(mdl)
    train_it(Model(inputs=inputs, outputs=mdl), n)
else:
    clf[n] = load_model('Classifier_HLF_v3/Classifier_'+n+'_v3.h5')

In [ ]:
n = 'Zprime'
if do_training:
    inputs = Input(shape=(original_dim,), name='Input_'+n)
    mdl = Dense(50, activation=act_fun, name='h1_'+n, kernel_constraint=max_norm(2.))(inputs)
#     mdl = Dropout(0.4)(mdl)
    mdl = Dense(50, activation=act_fun, name='h2_'+n, kernel_constraint=max_norm(2.))(mdl)
#     mdl = Dropout(0.4)(mdl)
    mdl = Dense(1, activation='sigmoid', name='out_'+n)(mdl)
    train_it(Model(inputs=inputs, outputs=mdl), n)
else:
    clf[n] = load_model('Classifier_'+n+'_v1.h5')

In [ ]:
n = 'Wprime'
if do_training:
    inputs = Input(shape=(original_dim,), name='Input_'+n)
    mdl = Dense(50, activation=act_fun, name='h1_'+n, kernel_constraint=max_norm(2.))(inputs)
#     mdl = Dropout(0.4)(mdl)
    mdl = Dense(50, activation=act_fun, name='h2_'+n, kernel_constraint=max_norm(2.))(mdl)
#     mdl = Dropout(0.5)(mdl)
    mdl = Dense(1, activation='sigmoid', name='out_'+n)(mdl)
    train_it(Model(inputs=inputs, outputs=mdl), n)
else:
    clf[n]= load_model('Classifier_'+n+'_v1.h5')

In [ ]:
n = 'leptoquark'
if do_training:
    intermediate_dim = 40
    inputs = Input(shape=(original_dim,), name='Input_'+n)
    mdl = Dense(intermediate_dim, activation=act_fun, name='h1_'+n, kernel_constraint=max_norm(2.))(inputs)
#     mdl = Dropout(0.3)(mdl)
    mdl = Dense(intermediate_dim, activation=act_fun, name='h2_'+n, kernel_constraint=max_norm(2.))(mdl)
#     mdl = Dropout(0.4)(mdl)
    mdl = Dense(1, activation='sigmoid', name='out_'+n)(mdl)
    train_it(Model(inputs=inputs, outputs=mdl), n)
else:
    clf[n] = load_model('Classifier_'+n+'_v1.h5')

In [ ]:
n = 'hToTauTau'
if do_training:
    intermediate_dim = 40
    inputs = Input(shape=(original_dim,), name='Input_'+n)
    mdl = Dense(intermediate_dim, activation=act_fun, name='h1_'+n, kernel_constraint=max_norm(2.))(inputs)
#     mdl = Dropout(0.3)(mdl)
    mdl = Dense(intermediate_dim, activation=act_fun, name='h2_'+n, kernel_constraint=max_norm(2.))(mdl)
#     mdl = Dropout(0.4)(mdl)
    mdl = Dense(1, activation='sigmoid', name='out_'+n)(mdl)
    train_it(Model(inputs=inputs, outputs=mdl), n)
else:
    clf[n] = load_model('Classifier_'+n+'_v1.h5')

In [ ]:
n = 'hChToTauNu'
if do_training:
    intermediate_dim = 40
    inputs = Input(shape=(original_dim,), name='Input_'+n)
    mdl = Dense(intermediate_dim, activation=act_fun, name='h1_'+n, kernel_constraint=max_norm(2.))(inputs)
#     mdl = Dropout(0.3)(mdl)
    mdl = Dense(intermediate_dim, activation=act_fun, name='h2_'+n, kernel_constraint=max_norm(2.))(mdl)
#     mdl = Dropout(0.4)(mdl)
    mdl = Dense(1, activation='sigmoid', name='out_'+n)(mdl)
    train_it(Model(inputs=inputs, outputs=mdl), n)
else:
    clf[n] = load_model('Classifier_'+n+'_v1.h5')

## Plot the ROC curves

In [ ]:
Eff_SM = 5.38e-06

In [ ]:
def ROC_curve(p_BSM, p_SM, eval_q_SM):
    eval_p = np.percentile(1-p_SM, q=100*eval_q_SM)
    
    out = (1-p_BSM) < eval_p
    out = np.sum(out, axis=0)
    q_BSM = out/float(p_BSM.shape[0])
    
    AUC = np.trapz(q_BSM, eval_q_SM)
    
    return q_BSM, AUC

In [ ]:
matplotlib.rcParams.update({'font.size': 16})

f, ax_arr = plt.subplots(3,2 , figsize=(18,18))

f_ROC, ax_arr_ROC = plt.subplots(1,1, figsize=(10,10))

for i,n in enumerate(BSM_samples):
    p_SM = clf[n].predict(x_val, batch_size=2000)
    
    l_bsm = int(x_sig[n].shape[0]*training_split_fraction)
    p_BSM = clf[n].predict(x_sig[n][l_bsm:], batch_size=2000)
    
    # print ROC curve
    q_SM = np.logspace(base=10, start=-7, stop=0, num=100)
    q_BSM, roc_auc = ROC_curve(p_BSM, p_SM, q_SM)
    dic2save = {'eff_BSM':q_BSM, 'eff_SM':q_SM, 'roc_auc':roc_auc}
    fileout = open('Classifier_HLF_v3/ROC_dict_{}.pkl'.format(n), 'w')
    pickle.dump(file=fileout, obj=dic2save)
    fileout.close()
    ax_arr_ROC.plot(q_SM, q_BSM, label='{} (area = {:0.2f})'.format(n, roc_auc))
    
    

    q = np.percentile(p_SM, 100*(1-Eff_SM))
    eff_BSM = float(np.sum(p_BSM>q))/x_sig[n][l_bsm:].shape[0]

    print '{}: {:1.2e}'.format(n, eff_BSM)
    
    bins = np.logspace(base=10, start=-5, stop=0, num=50)
    if n=='Wprime':
        bins = np.logspace(base=10, start=-2, stop=0, num=50)
    ax_arr[i/2,i%2].hist(1-p_SM, bins=bins, alpha = 0.5, label='SMMix')
    ax_arr[i/2,i%2].hist(1-p_BSM, bins=bins, alpha=0.5, label=n)
    ax_arr[i/2,i%2].plot([1-q,1-q], [0,1e6], '--r', label='cut')

    ax_arr[i/2,i%2].set_title('{} classifier, eff {:1.2e}'.format(n, eff_BSM))
    ax_arr[i/2,i%2].legend(loc='best')
    ax_arr[i/2,i%2].set_ylabel('Events')
    ax_arr[i/2,i%2].set_xlabel('Probability SM predicted')
    ax_arr[i/2,i%2].set_yscale('log')
    ax_arr[i/2,i%2].set_xscale('log')
    
ax_arr_ROC.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
ax_arr_ROC.plot([Eff_SM, Eff_SM], [1e-6, 1.05], color='red', lw=2, linestyle='--', label='1000 SM evts/month')
ax_arr_ROC.set_xlim([1e-6, 1.0])
ax_arr_ROC.set_ylim([1e-6, 1.05])
ax_arr_ROC.set_xlabel('SM efficiency')
ax_arr_ROC.set_ylabel('BSM efficiency')
ax_arr_ROC.set_title('ROC supervised training')
ax_arr_ROC.legend(loc="lower right")
ax_arr_ROC.set_yscale('log')
ax_arr_ROC.set_xscale('log')
ax_arr_ROC.grid()
f_ROC.savefig('Classifier_HLF_v3/ROC_clf.png')

In [ ]:
for k,c in clf.iteritems():
    c.save('Classifier_HLF_v3/Classifier_'+k+'_v3.h5')